notebook to test and find best hyperparameters for various models

In [7]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV as RSCV

In [3]:
import os

pid = os.getpid()
print("PID: %i" % pid)

n_cpu = os.cpu_count()   # Number of CPUs assigned to this process
print("Number of CPUs in the system:", n_cpu)

# we won't use all the available cpu's for this script 
n_jobs = n_cpu - 2 # The number of tasks to run in parallel

# Control which CPUs are made available for this script
cpu_arg = ''.join([str(ci) + ',' for ci in list(range(n_jobs))])[:-1]
cmd = 'taskset -cp %s %i' % (cpu_arg, pid)
print("executing command '%s' ..." % cmd)
os.system(cmd)

PID: 11708
Number of CPUs in the system: 16
executing command 'taskset -cp 0,1,2,3,4,5,6,7,8,9,10,11,12,13 11708' ...


1

In [33]:
# path to cleaned data file
trainFilePath = 'data/data_train.pkl'

# Read already prepared and saved train datasets
with open(trainFilePath, 'rb') as f:
    data_train = pickle.load(f)

# Create dummy variables for categorical data
data_train = pd.get_dummies(data_train, columns=['clinical_stage', 'biopsy_gleason_gg', 'pathological_gleason_gg',
                                'pathologic_stage', 'lni', 'surgical_margin_status', 'persistent_psa',
                                'TRYSgrupes', 'PLNDO1'])

In [5]:
"""
Explodes the provided "df" dataset based on provided survival column "time" and
clips the data to be in a range [min_time; max_time] (). A new discrete survival column
will be created with name set as variable "time_discrete". "cum_event" boolean determines
if cumulative event column will be created or no.

clip(lower, upper) function will help us create a new discrete survival time column. 
If we specify lower=1 and upper=200, patients who experienced event earlier than 200th 
month will only have records till their event, on other side, if a patient survived past 
200th month, we will clip this information and will only keep information about him til 200th month.
Another example, if we specify lower=140 and upper=200, and if the person experienced event 
at 100th month, we will create records for him till 140th (lower boundary) month.
"""
def explode_data(df,max_time,time,target_column,min_time=1,
                 time_discrete='survival_time_discrete',cum_event=False):

    target_column_discrete = target_column + '_discrete'

    # We create a new time column and clip the data by provided min and max survival times
    df[time_discrete] = df[time].clip(min_time,max_time).apply(range)

    # Exploding the dataset with the created range value in new time column
    data_exploded = df.explode(time_discrete)
    data_exploded.reset_index(drop=True, inplace=True)

    # New column starts at 0, we'll increase each value by 1
    data_exploded[time_discrete] = pd.to_numeric(data_exploded[time_discrete]) + 1

    # New event column, which will indicate the last event date
    data_exploded[target_column_discrete] = (data_exploded[time_discrete] >= data_exploded[time]) * pd.to_numeric(data_exploded[target_column])
    
    if cum_event == True:
        target_column_cumulative = target_column + '_cumulative'

        # Create new event column with duplicated event values from discrete column
        data_exploded[target_column_cumulative] = data_exploded[target_column_discrete]
        
        # For cumulative events, after end_time we will have NA values, we'll replace those with event indicator
        after_survival_time = data_exploded[time_discrete] > data_exploded[time]
        data_exploded.loc[after_survival_time, target_column_discrete] = -1
        data_exploded[target_column_discrete] = data_exploded[target_column_discrete].replace(-1,np.NaN)
        data_exploded.loc[(after_survival_time & (data_exploded[target_column]==0)), target_column_cumulative] = -1
        data_exploded[target_column_cumulative] = data_exploded[target_column_cumulative].replace(-1,np.NaN)

    return data_exploded


"""
Given an exploded dataset with instant mortality probabilities "event_probability_column"
and "id_column" for grouping (optional), cumulative hazard column will be calculated
"""
def cumulative_hazard(df, event_probability_column, id_column):
    data_copy = df.copy()
    if id_column is not None:
        data_copy = data_copy[ [id_column, event_probability_column] ]
    else:
        data_copy = data_copy[ [event_probability_column] ]
    data_copy['negative_log_prob'] = np.log( 1 - data_copy[event_probability_column] )
    if id_column is not None:
        data_copy['cumulative_hazard'] = 1 - np.exp(data_copy.groupby(id_column)['negative_log_prob'].transform(pd.Series.cumsum))
    else:
        data_copy['cumulative_hazard'] = 1 - np.exp(data_copy['negative_log_prob'].cumsum())
    return data_copy['cumulative_hazard']


"""
Given non exploded dataset, explodes the dataset based on "max_time", lower boundary for exploding
a dataset will be the same as upper, so for each patient we will create "max_time" records.
Adds predictent instant mortality probabilities as well as cumulative ones.
"""
def add_predict_probabilities(df, model, max_time, target_column, time, is_homogenous_dataset=False):

    df_exploded = explode_data(df, max_time=max_time, min_time=max_time, cum_event=True, time=time, target_column=target_column)

    x_columns_to_drop = [target_column, target_column+'_discrete', target_column+'_cumulative', 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
    X_df = df_exploded.drop(x_columns_to_drop, axis=1)

    # probabilities
    y_pred = model.predict_proba(X_df)[:,1]
    df_exploded['mortality_instant_prob'] = y_pred

    # Cumulative hazard for each patient
    df_exploded['cumulative_hazard'] = cumulative_hazard(df_exploded,'mortality_instant_prob','patient_id')
    
    # TODO - homogenous dataset
    # if is_homogenous_dataset:
        
    #     # get adjustment parameters
    #     match target_column:
    #         case 'cancer_specific_mortality':
    #             pi_0 = pi_0_csm
    #             pi_1 = pi_1_csm
    #             rho_0 = rho_0_csm
    #             rho_1 = rho_1_csm
    #         case 'death_from_other_causes':
    #             pi_0 = pi_0_doc
    #             pi_1 = pi_1_doc
    #             rho_0 = rho_0_doc
    #             rho_1 = rho_1_doc
    #         case 'mts':
    #             pi_0 = pi_0_mts
    #             pi_1 = pi_1_mts
    #             rho_0 = rho_0_mts
    #             rho_1 = rho_1_mts
    #         case 'bcr':
    #             pi_0 = pi_0_bcr
    #             pi_1 = pi_1_bcr
    #             rho_0 = rho_0_bcr
    #             rho_1 = rho_1_bcr
    #         case _:
    #             pi_0 = None
    #             pi_1 = None
    #             rho_0 = None
    #             rho_1 = None
        
        
    #     # adjusted probabilities
    #     df_exploded['mortality_instant_prob_adjusted'] = \
    #         (df_exploded.mortality_instant_prob*(pi_1/rho_1)) / \
    #         ((1-df_exploded.mortality_instant_prob)*(pi_0/rho_0) + \
    #          df_exploded.mortality_instant_prob*(pi_1/rho_1))
        
    #     # Cumulative hazard for each patient (adjusted)
    #     df_exploded['cumulative_hazard_adjusted'] = cumulative_hazard(df_exploded,'mortality_instant_prob_adjusted','patient_id')
    
    return df_exploded

## Random Forest

### RandomizedSearchCV

#### Cancer specific mortality

In [11]:
target_column = 'cancer_specific_mortality'
max_time = 200

# mts and bcr have different survival months columns
match target_column:
    case 'mts':
        time = 'survival_months_mts'
    case 'bcr':
        time = 'survival_months_bcr'
    case _:
        time = 'survival_months'

target_column_discrete = target_column + '_discrete'

# List of columns names which will be dropped from feature set before fitting the model
target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

# Explode the dataset
df_train_copy_exploded = explode_data(data_train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

# Drop targets/features from feature set
x_columns_to_drop = [target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
x_columns_to_drop.extend(target_columns)
X_train = df_train_copy_exploded.drop(x_columns_to_drop, axis=1)    
y_train = df_train_copy_exploded[target_column_discrete]

In [13]:
X_train.columns

Index(['age', 'psa', 'biopsy_gleason', 'pathologic_gleason',
       'clinical_stage_1', 'clinical_stage_2', 'clinical_stage_3',
       'biopsy_gleason_gg_1', 'biopsy_gleason_gg_2', 'biopsy_gleason_gg_3',
       'biopsy_gleason_gg_4', 'biopsy_gleason_gg_5',
       'pathological_gleason_gg_1', 'pathological_gleason_gg_2',
       'pathological_gleason_gg_3', 'pathological_gleason_gg_4',
       'pathological_gleason_gg_5', 'pathologic_stage_0', 'pathologic_stage_1',
       'pathologic_stage_2', 'lni_0.0', 'lni_1.0', 'lni_unknown',
       'surgical_margin_status_0', 'surgical_margin_status_1',
       'persistent_psa_0', 'persistent_psa_1', 'TRYSgrupes_0', 'TRYSgrupes_1',
       'TRYSgrupes_2', 'PLNDO1_0', 'PLNDO1_1', 'survival_time_discrete'],
      dtype='object')

In [21]:
param_grid = {'n_estimators':np.arange(150,700,50),
              'max_features':np.arange(0.1, 1, 0.1),
              'max_depth': [3, 5, 7, 9],
              'max_samples': [0.3, 0.5, 0.8],
              'min_samples_leaf': [1, 2, 3, 4, 5]}

rf_random = RSCV(RandomForestClassifier(), param_grid, n_iter=15, 
             random_state=0, verbose=2, scoring='roc_auc', n_jobs=n_jobs)
model_rf = rf_random.fit(X_train, y_train)
model_rf_best = model_rf.best_estimator_

Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [22]:
# Best parameters
model_rf.best_params_

{'n_estimators': 200,
 'min_samples_leaf': 5,
 'max_samples': 0.3,
 'max_features': 0.5,
 'max_depth': 3}

### Bayesian Optimization

#### cancer specific mortality

In [46]:
target_column = 'cancer_specific_mortality'
max_time = 200

# mts and bcr have different survival months columns
match target_column:
    case 'mts':
        time = 'survival_months_mts'
    case 'bcr':
        time = 'survival_months_bcr'
    case _:
        time = 'survival_months'

target_column_discrete = target_column + '_discrete'

# List of columns names which will be dropped from feature set before fitting the model
target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

# Explode the dataset
df_train_copy_exploded = explode_data(data_train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

# Drop targets/features from feature set
x_columns_to_drop = [target_column, target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
x_columns_to_drop.extend(target_columns)
X_train = df_train_copy_exploded.drop(x_columns_to_drop, axis=1)    
y_train = df_train_copy_exploded[target_column_discrete]

In [68]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
from sklearn.model_selection import cross_val_score

space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = int(space['max_depth']),
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = int(space['n_estimators']), 
                                 )
    
    model.fit(X_train, y_train)

    #auc = cross_val_score(model, X_train, y_train, cv = 5, scoring='roc_auc').mean()
    # Test on training data
    data_train_copy = data_train.copy()
    target_columns_to_drop = target_columns.copy()
    target_columns_to_drop.remove(target_column)
    data_train_copy = data_train_copy.drop(target_columns_to_drop, axis=1)
    df_train_predicted = add_predict_probabilities(data_train_copy, model=model, max_time=max_time, 
                                                     target_column=target_column, time=time)

    # AUC for each cumulative slice
    # Months at which we'll check the AUC's
    months = list(range(6, max_time, 6))

    train_auc_stats = []
    for month in months:
        # --- Training data ---
        # Selecting a subset of data based on the months
        select = (df_train_predicted['survival_time_discrete'] == month) & pd.notna(df_train_predicted[target_column+'_cumulative'])
        sub_dat = df_train_predicted[select]

        # If in the sliced data there's a event, calculate AUC metric,
        # otherwise assign NaN value
        if sub_dat[target_column+'_cumulative'].max() == 1:
            fpr, tpr, thresholds = roc_curve(sub_dat[target_column+'_cumulative'], sub_dat['cumulative_hazard'])
            auc_stat = auc(fpr, tpr)
        else:
            auc_stat = float('NaN')
        train_auc_stats.append(auc_stat)

    auc_mean = np.nanmean(train_auc_stats)

    # We aim to maximize auc, therefore we return it as a negative value
    return {'loss': -auc_mean, 'status': STATUS_OK }

In [70]:
trials = Trials()
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 40, #80
            trials = trials)
best

100%|██████████| 40/40 [11:22<00:00, 17.07s/trial, best loss: -0.9258064473457128]


{'criterion': 1,
 'max_depth': 18.61491475813544,
 'max_features': 2,
 'min_samples_leaf': 5.4715419273109606e-05,
 'min_samples_split': 0.3178996881755699,
 'n_estimators': 239.84475133029923}

In [29]:
best

{'criterion': 1,
 'max_depth': 200.0,
 'max_features': 3,
 'min_samples_leaf': 0.043813847274908446,
 'min_samples_split': 0.551636834827235,
 'n_estimators': 5}